<a href="https://colab.research.google.com/github/xjdeng/youtube-ai-query/blob/main/ai_query_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from google.colab import userdata
import google.generativeai as genai
import json
import pprint
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') #Set up Gemini Credentials, see video: https://www.youtube.com/watch?v=S1elvCs1gyI
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")

In [ ]:
def get_transcript(video_url):
  if "&" in video_url:
    video_url = video_url.split("&")[0]
  video_id = video_url.split("v=")[1]
  entries = []
  try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id)
      for entry in transcript:
          entries.append(entry)
  except Exception as e:
      print("Error:", e)
  return entries

def get_transcripts(video_url_list):
  if isinstance(video_url_list, str):
    video_url_list = [video_url_list]
  transcripts = []
  for video_url in video_url_list:
    transcript = get_transcript(video_url)
    if len(transcript) > 0:
      transcripts.append({"Video": video_url, "Transcript": transcript})
  print(f"Got {len(transcripts)} transcripts")
  return transcripts

In [ ]:
def query_youtubes(query, video_url_list):
  transcripts = get_transcripts(video_url_list)
  prompt = f"""

  I'd like to ask the following question on a list of Youtube transcripts I'm providing. They may or may not be from different Youtubers.

  Here's the question:
  ---
  {query}
  ---

  Do not download data from Internet, instead, formulate your answer using the following Youtube transcripts:
  ---
  {transcripts}
  ---

  """
  response = model.generate_content(prompt)
  return response.text

In [ ]:
videos= [
    "https://www.youtube.com/watch?v=y3Umo_jd5AA&pp=ygUXQ2FsIE5ld3BvcnQgbGV4IGZyaWRtYW4%3D",
    "https://www.youtube.com/watch?v=atuEOcRznpA&pp=ygUWQ2FsIE5ld3BvcnQgdGltIGZlcnJpcw%3D%3D",
    "https://www.youtube.com/watch?v=z6IgPEO2jAk&pp=ygUcQ2FsIE5ld3BvcnQgam9yZGFuIGhhcmJpbmdlcg%3D%3D",

]

query = "What do they think about pizza?"

pprint.pprint(query_youtubes(query, videos))

In [ ]:
# @title
from google.colab import files
import json
transcripts = get_transcripts(videos)
fname = "transcripts.jsonl"
json.dump(transcripts, open(fname, "w"))
files.download(fname)